In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Configurar estilo
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (15, 10)

# Ler os dados
data = """pdf_size_mb,algo,tempo_local_ms,ciclos_local,tempo_intel_x9_ms,tempo_amd_e6_ms,custo_intel_r$,custo_amd_r$
[seus dados aqui]"""

# Como você tem os dados em um arquivo, use:
df = pd.read_csv('benchmark_dsa.csv')

# Ou se quiser colar os dados diretamente:
# from io import StringIO
# df = pd.read_csv(StringIO(data))

# Criar figura com múltiplos subplots
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Análise de Performance - ML-DSA Algorithms', fontsize=16, fontweight='bold')

# 1. Tempo de execução por tamanho de PDF
ax1 = axes[0, 0]
for algo in df['algo'].unique():
    data_algo = df[df['algo'] == algo]
    ax1.plot(data_algo['pdf_size_mb'], data_algo['tempo_local_ms'], 
             marker='o', label=algo, linewidth=2, markersize=4, alpha=0.7)
ax1.set_xlabel('Tamanho do PDF (MB)', fontsize=11)
ax1.set_ylabel('Tempo Local (ms)', fontsize=11)
ax1.set_title('Tempo de Execução por Tamanho de PDF', fontsize=12, fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

# 2. Comparação de custos entre Intel e AMD
ax2 = axes[0, 1]
for algo in df['algo'].unique():
    data_algo = df[df['algo'] == algo]
    ax2.plot(data_algo['pdf_size_mb'], data_algo['custo_intel_r$'], 
             label=f'{algo} (Intel)', linewidth=2, alpha=0.7)
    ax2.plot(data_algo['pdf_size_mb'], data_algo['custo_amd_r$'], 
             label=f'{algo} (AMD)', linewidth=2, linestyle='--', alpha=0.7)
ax2.set_xlabel('Tamanho do PDF (MB)', fontsize=11)
ax2.set_ylabel('Custo (R$)', fontsize=11)
ax2.set_title('Comparação de Custos: Intel vs AMD', fontsize=12, fontweight='bold')
ax2.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=8)
ax2.grid(True, alpha=0.3)

# 3. Ciclos de CPU por algoritmo
ax3 = axes[1, 0]
for algo in df['algo'].unique():
    data_algo = df[df['algo'] == algo]
    ax3.scatter(data_algo['pdf_size_mb'], data_algo['ciclos_local']/1e9, 
                label=algo, alpha=0.6, s=30)
ax3.set_xlabel('Tamanho do PDF (MB)', fontsize=11)
ax3.set_ylabel('Ciclos de CPU (Bilhões)', fontsize=11)
ax3.set_title('Ciclos de CPU por Algoritmo', fontsize=12, fontweight='bold')
ax3.legend()
ax3.grid(True, alpha=0.3)

# 4. Comparação de tempo: Intel vs AMD
ax4 = axes[1, 1]
for algo in df['algo'].unique():
    data_algo = df[df['algo'] == algo]
    diferenca = ((data_algo['tempo_intel_x9_ms'] - data_algo['tempo_amd_e6_ms']) / 
                 data_algo['tempo_amd_e6_ms'] * 100)
    ax4.plot(data_algo['pdf_size_mb'], diferenca, 
             marker='o', label=algo, linewidth=2, markersize=4, alpha=0.7)
ax4.axhline(y=0, color='black', linestyle='--', linewidth=1, alpha=0.5)
ax4.set_xlabel('Tamanho do PDF (MB)', fontsize=11)
ax4.set_ylabel('Diferença (%)', fontsize=11)
ax4.set_title('Intel X9 vs AMD E6 - Diferença de Performance (%)', fontsize=12, fontweight='bold')
ax4.legend()
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('analise_ml_dsa.png', dpi=300, bbox_inches='tight')
plt.show()

# Gráfico adicional: Boxplot de custos por algoritmo
fig2, ax = plt.subplots(1, 2, figsize=(14, 6))

# Custos Intel
df_intel = df[['algo', 'custo_intel_r$']].copy()
df_intel['plataforma'] = 'Intel X9'
df_intel.columns = ['algo', 'custo', 'plataforma']

# Custos AMD
df_amd = df[['algo', 'custo_amd_r$']].copy()
df_amd['plataforma'] = 'AMD E6'
df_amd.columns = ['algo', 'custo', 'plataforma']

df_custos = pd.concat([df_intel, df_amd])

sns.boxplot(data=df_custos, x='algo', y='custo', hue='plataforma', ax=ax[0])
ax[0].set_title('Distribuição de Custos por Algoritmo e Plataforma', fontsize=12, fontweight='bold')
ax[0].set_xlabel('Algoritmo', fontsize=11)
ax[0].set_ylabel('Custo (R$)', fontsize=11)
ax[0].tick_params(axis='x', rotation=45)

# Eficiência: Custo por MB
df['custo_intel_por_mb'] = df['custo_intel_r$'] / df['pdf_size_mb']
df['custo_amd_por_mb'] = df['custo_amd_r$'] / df['pdf_size_mb']

for algo in df['algo'].unique():
    data_algo = df[df['algo'] == algo]
    ax[1].plot(data_algo['pdf_size_mb'], data_algo['custo_intel_por_mb'], 
               label=f'{algo} (Intel)', linewidth=2, alpha=0.7)
    ax[1].plot(data_algo['pdf_size_mb'], data_algo['custo_amd_por_mb'], 
               label=f'{algo} (AMD)', linewidth=2, linestyle='--', alpha=0.7)

ax[1].set_title('Custo por MB de PDF', fontsize=12, fontweight='bold')
ax[1].set_xlabel('Tamanho do PDF (MB)', fontsize=11)
ax[1].set_ylabel('Custo/MB (R$)', fontsize=11)
ax[1].legend(fontsize=8)
ax[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('analise_custos_ml_dsa.png', dpi=300, bbox_inches='tight')
plt.show()

# Estatísticas resumidas
print("\n=== ESTATÍSTICAS RESUMIDAS ===\n")
print("Tempo médio por algoritmo (ms):")
print(df.groupby('algo')['tempo_local_ms'].mean().round(2))
print("\nCusto médio Intel por algoritmo (R$):")
print(df.groupby('algo')['custo_intel_r$'].mean().round(5))
print("\nCusto médio AMD por algoritmo (R$):")
print(df.groupby('algo')['custo_amd_r$'].mean().round(5))

FileNotFoundError: [Errno 2] No such file or directory: 'seu_arquivo.csv'